# Batch Normalization
One way to make deep networks easier to train is to use more sophisticated optimization procedures such as SGD+momentum, RMSProp, or Adam. Another strategy is to change the architecture of the network to make it easier to train. 
One idea along these lines is batch normalization which was proposed by [1] in 2015.

The idea is relatively straightforward. Machine learning methods tend to work better when their input data consists of uncorrelated features with zero mean and unit variance. When training a neural network, we can preprocess the data before feeding it to the network to explicitly decorrelate its features; this will ensure that the first layer of the network sees data that follows a nice distribution. However, even if we preprocess the input data, the activations at deeper layers of the network will likely no longer be decorrelated and will no longer have zero mean or unit variance since they are output from earlier layers in the network. Even worse, during the training process the distribution of features at each layer of the network will shift as the weights of each layer are updated.

The authors of [1] hypothesize that the shifting distribution of features inside deep neural networks may make training deep networks more difficult. To overcome this problem, [1] proposes to insert batch normalization layers into the network. At training time, a batch normalization layer uses a minibatch of data to estimate the mean and standard deviation of each feature. These estimated means and standard deviations are then used to center and normalize the features of the minibatch. A running average of these means and standard deviations is kept during training, and at test time these running averages are used to center and normalize features.

It is possible that this normalization strategy could reduce the representational power of the network, since it may sometimes be optimal for certain layers to have features that are not zero-mean or unit variance. To this end, the batch normalization layer includes learnable shift and scale parameters for each feature dimension.

[1] [Sergey Ioffe and Christian Szegedy, "Batch Normalization: Accelerating Deep Network Training by Reducing
Internal Covariate Shift", ICML 2015.](https://arxiv.org/abs/1502.03167)

In [1]:
# As usual, a bit of setup
import time
import numpy as np
import matplotlib.pyplot as plt
from cs231n.classifiers.fc_net import *
from cs231n.data_utils import get_CIFAR10_data
from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from cs231n.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

def print_mean_std(x,axis=0):
    print('  means: ', x.mean(axis=axis))
    print('  stds:  ', x.std(axis=axis))
    print() 

In [2]:
# Load the (preprocessed) CIFAR10 data.
data = get_CIFAR10_data()
for k, v in data.items():
  print('%s: ' % k, v.shape)

X_train:  (49000, 3, 32, 32)
y_train:  (49000,)
X_val:  (1000, 3, 32, 32)
y_val:  (1000,)
X_test:  (1000, 3, 32, 32)
y_test:  (1000,)


## Batch normalization: forward
In the file `cs231n/layers.py`, implement the batch normalization forward pass in the function `batchnorm_forward`. Once you have done so, run the following to test your implementation.

Referencing the paper linked to above in [1] may be helpful!

In [3]:
# Check the training-time forward pass by checking means and variances
# of features both before and after batch normalization   

# Simulate the forward pass for a two-layer network
np.random.seed(231)
N, D1, D2, D3 = 200, 50, 60, 3
X = np.random.randn(N, D1)
W1 = np.random.randn(D1, D2)
W2 = np.random.randn(D2, D3)
a = np.maximum(0, X.dot(W1)).dot(W2)

print('Before batch normalization:')
print_mean_std(a,axis=0)

gamma = np.ones((D3,))
beta = np.zeros((D3,))
# Means should be close to zero and stds close to one
print('After batch normalization (gamma=1, beta=0)')
a_norm, _ = batchnorm_forward(a, gamma, beta, {'mode': 'train'})
print_mean_std(a_norm,axis=0)

gamma = np.asarray([1.0, 2.0, 3.0])
beta = np.asarray([11.0, 12.0, 13.0])
# Now means should be close to beta and stds close to gamma
print('After batch normalization (gamma=', gamma, ', beta=', beta, ')')
a_norm, _ = batchnorm_forward(a, gamma, beta, {'mode': 'train'})
print_mean_std(a_norm,axis=0)

Before batch normalization:
  means:  [ -2.3814598  -13.18038246   1.91780462]
  stds:   [27.18502186 34.21455511 37.68611762]

After batch normalization (gamma=1, beta=0)
  means:  [2.22044605e-17 8.16013923e-17 4.57966998e-17]
  stds:   [0.99999999 1.         1.        ]

After batch normalization (gamma= [1. 2. 3.] , beta= [11. 12. 13.] )
  means:  [11. 12. 13.]
  stds:   [0.99999999 1.99999999 2.99999999]



In [4]:
# Check the test-time forward pass by running the training-time
# forward pass many times to warm up the running averages, and then
# checking the means and variances of activations after a test-time
# forward pass.

np.random.seed(231)
N, D1, D2, D3 = 200, 50, 60, 3
W1 = np.random.randn(D1, D2)
W2 = np.random.randn(D2, D3)

bn_param = {'mode': 'train'}
gamma = np.ones(D3)
beta = np.zeros(D3)

for t in range(50):
  X = np.random.randn(N, D1)
  a = np.maximum(0, X.dot(W1)).dot(W2)
  batchnorm_forward(a, gamma, beta, bn_param)

bn_param['mode'] = 'test'
X = np.random.randn(N, D1)
a = np.maximum(0, X.dot(W1)).dot(W2)
a_norm, _ = batchnorm_forward(a, gamma, beta, bn_param)

# Means should be close to zero and stds close to one, but will be
# noisier than training-time forward passes.
print('After batch normalization (test-time):')
print_mean_std(a_norm,axis=0)

After batch normalization (test-time):
  means:  [-0.03927354 -0.04349152 -0.10452688]
  stds:   [1.01531428 1.01238373 0.97819988]



## Batch normalization: backward
Now implement the backward pass for batch normalization in the function `batchnorm_backward`.

To derive the backward pass you should write out the computation graph for batch normalization and backprop through each of the intermediate nodes. Some intermediates may have multiple outgoing branches; make sure to sum gradients across these branches in the backward pass.

Once you have finished, run the following to numerically check your backward pass.

In [5]:
# Gradient check batchnorm backward pass
np.random.seed(231)
N, D = 4, 5
x = 5 * np.random.randn(N, D) + 12
gamma = np.random.randn(D)
beta = np.random.randn(D)
dout = np.random.randn(N, D)

bn_param = {'mode': 'train'}
fx = lambda x: batchnorm_forward(x, gamma, beta, bn_param)[0]
fg = lambda a: batchnorm_forward(x, a, beta, bn_param)[0]
fb = lambda b: batchnorm_forward(x, gamma, b, bn_param)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
da_num = eval_numerical_gradient_array(fg, gamma.copy(), dout)
db_num = eval_numerical_gradient_array(fb, beta.copy(), dout)

_, cache = batchnorm_forward(x, gamma, beta, bn_param)
dx, dgamma, dbeta = batchnorm_backward(dout, cache)
#You should expect to see relative errors between 1e-13 and 1e-8
print('dx error: ', rel_error(dx_num, dx))
print('dgamma error: ', rel_error(da_num, dgamma))
print('dbeta error: ', rel_error(db_num, dbeta))
print('\n', dx_num, '\n','\n', dx)


dx error:  1.7029261167605239e-09
dgamma error:  7.420414216247087e-13
dbeta error:  2.8795057655839487e-12

 [[-0.00310319  0.00305468 -0.00156246  0.17251307  0.01388029]
 [ 0.01147762 -0.10800884 -0.01112564 -0.02021632 -0.02098085]
 [-0.01682492 -0.01106847 -0.00384286  0.13581055 -0.04108612]
 [ 0.00845049  0.11602263  0.01653096 -0.2881073   0.04818669]] 
 
 [[-0.00310319  0.00305468 -0.00156246  0.17251307  0.01388029]
 [ 0.01147762 -0.10800884 -0.01112564 -0.02021632 -0.02098085]
 [-0.01682492 -0.01106847 -0.00384286  0.13581055 -0.04108612]
 [ 0.00845049  0.11602263  0.01653096 -0.2881073   0.04818669]]


## Batch normalization: alternative backward
In class we talked about two different implementations for the sigmoid backward pass. One strategy is to write out a computation graph composed of simple operations and backprop through all intermediate values. Another strategy is to work out the derivatives on paper. For example, you can derive a very simple formula for the sigmoid function's backward pass by simplifying gradients on paper.

Surprisingly, it turns out that you can do a similar simplification for the batch normalization backward pass too!  

In the forward pass, given a set of inputs $X=\begin{bmatrix}x_1\\x_2\\...\\x_N\end{bmatrix}$, 

we first calculate the mean $\mu$ and variance $v$.
With $\mu$ and $v$ calculated, we can calculate the standard deviation $\sigma$  and normalized data $Y$.
The equations and graph illustration below describe the computation ($y_i$ is the i-th element of the vector $Y$).

\begin{align}
& \mu=\frac{1}{N}\sum_{k=1}^N x_k  &  v=\frac{1}{N}\sum_{k=1}^N (x_k-\mu)^2 \\
& \sigma=\sqrt{v+\epsilon}         &  y_i=\frac{x_i-\mu}{\sigma}
\end{align}

<img src="notebook_images/batchnorm_graph.png" width=691 height=202>

The meat of our problem during backpropagation is to compute $\frac{\partial L}{\partial X}$, given the upstream gradient we receive, $\frac{\partial L}{\partial Y}.$ To do this, recall the chain rule in calculus gives us $\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y} \cdot \frac{\partial Y}{\partial X}$.

The unknown/hart part is $\frac{\partial Y}{\partial X}$. We can find this by first deriving step-by-step our local gradients at 
$\frac{\partial v}{\partial X}$, $\frac{\partial \mu}{\partial X}$,
$\frac{\partial \sigma}{\partial v}$, 
$\frac{\partial Y}{\partial \sigma}$, and $\frac{\partial Y}{\partial \mu}$,
and then use the chain rule to compose these gradients (which appear in the form of vectors!) appropriately to compute $\frac{\partial Y}{\partial X}$.

If it's challenging to directly reason about the gradients over $X$ and $Y$ which require matrix multiplication, try reasoning about the gradients in terms of individual elements $x_i$ and $y_i$ first: in that case, you will need to come up with the derivations for $\frac{\partial L}{\partial x_i}$, by relying on the Chain Rule to first calculate the intermediate $\frac{\partial \mu}{\partial x_i}, \frac{\partial v}{\partial x_i}, \frac{\partial \sigma}{\partial x_i},$ then assemble these pieces to calculate $\frac{\partial y_i}{\partial x_i}$. 

You should make sure each of the intermediary gradient derivations are all as simplified as possible, for ease of implementation. 

After doing so, implement the simplified batch normalization backward pass in the function `batchnorm_backward_alt` and compare the two implementations by running the following. Your two implementations should compute nearly identical results, but the alternative implementation should be a bit faster.

In [6]:
np.random.seed(231)
N, D = 100, 500
x = 5 * np.random.randn(N, D) + 12
gamma = np.random.randn(D)
beta = np.random.randn(D)
dout = np.random.randn(N, D)

bn_param = {'mode': 'train'}
out, cache = batchnorm_forward(x, gamma, beta, bn_param)

t1 = time.time()
dx1, dgamma1, dbeta1 = batchnorm_backward(dout, cache)
t2 = time.time()
dx2, dgamma2, dbeta2 = batchnorm_backward_alt(dout, cache)
t3 = time.time()

print('dx difference: ', rel_error(dx1, dx2))
print('dgamma difference: ', rel_error(dgamma1, dgamma2))
print('dbeta difference: ', rel_error(dbeta1, dbeta2))
print('speedup: %.2fx' % ((t2 - t1) / (t3 - t2)))

#print(dx1,'\n','\n', dx2)

dx difference:  6.117609279344672e-13
dgamma difference:  0.0
dbeta difference:  0.0
speedup: 3.90x


## Fully Connected Nets with Batch Normalization
Now that you have a working implementation for batch normalization, go back to your `FullyConnectedNet` in the file `cs231n/classifiers/fc_net.py`. Modify your implementation to add batch normalization.

Concretely, when the `normalization` flag is set to `"batchnorm"` in the constructor, you should insert a batch normalization layer before each ReLU nonlinearity. The outputs from the last layer of the network should not be normalized. Once you are done, run the following to gradient-check your implementation.

HINT: You might find it useful to define an additional helper layer similar to those in the file `cs231n/layer_utils.py`. If you decide to do so, do it in the file `cs231n/classifiers/fc_net.py`.

In [7]:
np.random.seed(231)
N, D, H1, H2, C = 2, 15, 20, 30, 10
X = np.random.randn(N, D)
y = np.random.randint(C, size=(N,))

# You should expect losses between 1e-4~1e-10 for W, 
# losses between 1e-08~1e-10 for b,
# and losses between 1e-08~1e-09 for beta and gammas.
for reg in [0, 3.14]:
  print('Running check with reg = ', reg)
  model = FullyConnectedNet([H1, H2], input_dim=D, num_classes=C,
                            reg=reg, weight_scale=5e-2, dtype=np.float64,
                            normalization='batchnorm')

  loss, grads = model.loss(X, y)
  print('Initial loss: ', loss)

  for name in sorted(grads):
    f = lambda _: model.loss(X, y)[0]
    grad_num = eval_numerical_gradient(f, model.params[name], verbose=False, h=1e-5)
    print('%s relative error: %.2e' % (name, rel_error(grad_num, grads[name])))
  if reg == 0: print()

Running check with reg =  0
batchnorm
{'mode': 'train'}
Initial loss:  2.2611955101340957
batchnorm
{'mode': 'train', 'running_mean': array([ 8.93078480e-05,  4.40487007e-03, -1.63480491e-03,  1.68972923e-02,
       -1.77097920e-02,  1.02940765e-02,  3.08216864e-03,  2.94848771e-03,
        5.94064987e-03,  1.28799624e-02,  7.03992887e-04,  7.57920429e-03,
       -1.03487178e-02, -5.51334196e-03, -1.25779236e-02,  3.91239217e-03,
       -8.34741041e-03, -4.39992891e-03, -9.47560639e-03,  1.62992661e-02]), 'running_var': array([4.57587110e-07, 8.05116849e-05, 4.56248766e-05, 1.47632423e-03,
       7.08710729e-03, 8.92615349e-04, 1.12469252e-03, 3.27459418e-04,
       6.48905461e-04, 6.01974250e-05, 4.51509111e-04, 4.14788005e-04,
       1.60325229e-03, 1.52821293e-03, 1.63534651e-03, 3.96785633e-04,
       3.44711147e-04, 2.68677296e-07, 4.54817133e-04, 4.25744119e-04])}
batchnorm
{'mode': 'train', 'running_mean': array([ 0.00016968,  0.00836925, -0.00310613,  0.03210486, -0.0336486 ,
 

batchnorm
{'mode': 'train', 'running_mean': array([ 0.00089307,  0.04404794, -0.01634776,  0.16896999, -0.17709482,
        0.10293899,  0.03082117,  0.02948439,  0.05940549,  0.12879742,
        0.00703985,  0.07579072, -0.10348538, -0.05513246, -0.12577705,
        0.03912324, -0.08347265, -0.04399853, -0.09475442,  0.16298981]), 'running_var': array([4.57586449e-06, 8.05101693e-04, 4.56241871e-04, 1.47629924e-02,
       7.08698179e-02, 8.92598921e-03, 1.12467413e-02, 3.27454520e-03,
       6.48892753e-03, 6.01969058e-04, 4.51503028e-03, 4.14780795e-03,
       1.60322439e-02, 1.52818635e-02, 1.63531808e-02, 3.96778759e-03,
       3.44705188e-03, 2.68674322e-06, 4.54809124e-03, 4.25736601e-03])}
batchnorm
{'mode': 'train', 'running_mean': array([ 0.00089307,  0.04404802, -0.01634779,  0.16897028, -0.17709513,
        0.10293916,  0.03082122,  0.02948444,  0.05940559,  0.12879764,
        0.00703986,  0.0757904 , -0.10348556, -0.05513256, -0.12577727,
        0.03912331, -0.0834728 , -

       3.44710686e-03, 2.68661873e-06, 4.54809955e-03, 4.25744130e-03])}
batchnorm
{'mode': 'train', 'running_mean': array([ 0.00089308,  0.0440487 , -0.01634805,  0.16897292, -0.17709792,
        0.10294076,  0.03082168,  0.02948487,  0.05940649,  0.12879962,
        0.00703992,  0.07579203, -0.10348719, -0.05513344, -0.12577926,
        0.0391239 , -0.08347413, -0.04399933, -0.09475611,  0.16299266]), 'running_var': array([4.57587642e-06, 8.05116763e-04, 4.56248845e-04, 1.47632429e-02,
       7.08710713e-02, 8.92615283e-03, 1.12469261e-02, 3.27459479e-03,
       6.48905355e-03, 6.01974647e-04, 4.51509245e-03, 4.14787846e-03,
       1.60325190e-02, 1.52821246e-02, 1.63534592e-02, 3.96785271e-03,
       3.44710732e-03, 2.68663415e-06, 4.54817861e-03, 4.25744129e-03])}
batchnorm
{'mode': 'train', 'running_mean': array([ 0.00089308,  0.0440487 , -0.01634805,  0.16897292, -0.17709792,
        0.10294076,  0.03082168,  0.02948487,  0.05940649,  0.12879962,
        0.00703992,  0.07579203, 

       3.44711125e-03, 2.68676545e-06, 4.54817172e-03, 4.25744165e-03])}
batchnorm
{'mode': 'train', 'running_mean': array([ 0.00089306,  0.04404868, -0.01634808,  0.16897289, -0.17709796,
        0.10294071,  0.03082162,  0.0294848 ,  0.0594064 ,  0.12879962,
        0.00703993,  0.07579204, -0.10348718, -0.05513342, -0.12577924,
        0.03912392, -0.0834741 , -0.04399929, -0.09475606,  0.16299266]), 'running_var': array([4.57580583e-06, 8.05117963e-04, 4.56247737e-04, 1.47632351e-02,
       7.08710925e-02, 8.92616210e-03, 1.12469133e-02, 3.27458625e-03,
       6.48906842e-03, 6.01974269e-04, 4.51509117e-03, 4.14787997e-03,
       1.60325227e-02, 1.52821290e-02, 1.63534648e-02, 3.96785615e-03,
       3.44711128e-03, 2.68676620e-06, 4.54817168e-03, 4.25744161e-03])}
batchnorm
{'mode': 'train', 'running_mean': array([ 0.00089306,  0.04404868, -0.01634807,  0.16897289, -0.17709796,
        0.10294072,  0.03082163,  0.0294848 ,  0.05940641,  0.12880063,
        0.00703993,  0.07579204, 

{'mode': 'train', 'running_mean': array([ 0.00089308,  0.0440487 , -0.01634805,  0.16897292, -0.17709792,
        0.10294077,  0.03082169,  0.02948488,  0.0594065 ,  0.12879963,
        0.00703993,  0.07579204, -0.10348718, -0.05513342, -0.12577924,
        0.03912392, -0.0834741 , -0.04399929, -0.09475606,  0.16299266]), 'running_var': array([4.57589204e-06, 8.05116512e-04, 4.56249080e-04, 1.47632445e-02,
       7.08710669e-02, 8.92615088e-03, 1.12469288e-02, 3.27459659e-03,
       6.48905043e-03, 6.01975827e-04, 4.51509139e-03, 4.14787972e-03,
       1.60325221e-02, 1.52821283e-02, 1.63534639e-02, 3.96785557e-03,
       3.44711060e-03, 2.68674494e-06, 4.54817286e-03, 4.25744302e-03])}
batchnorm
{'mode': 'train', 'running_mean': array([ 0.00089308,  0.0440487 , -0.01634805,  0.16897292, -0.17709792,
        0.10294077,  0.03082169,  0.02948488,  0.0594065 ,  0.12879963,
        0.00703986,  0.07579204, -0.10348718, -0.05513342, -0.12577924,
        0.03912392, -0.0834741 , -0.04399929

...

batchnorm
{'mode': 'train', 'running_mean': array([-0.08286335, -0.07727974,  0.08525366, -0.12282139, -0.2330962 ,
       -0.13138105, -0.03352845,  0.04514041, -0.06611079, -0.12770451,
       -0.03460573,  0.11581767, -0.04147041, -0.04217178,  0.07648135,
        0.04472147, -0.03398877,  0.26535478, -0.07140276, -0.1518474 ]), 'running_var': array([0.00071735, 0.01194891, 0.02134588, 0.00237688, 0.00552786,
       0.00140453, 0.00142502, 0.0182396 , 0.0021617 , 0.06670861,
       0.01578707, 0.02765941, 0.00023354, 0.00146228, 0.02689178,
       0.01527322, 0.01009503, 0.01312208, 0.01002459, 0.02365058])}
batchnorm
{'mode': 'train', 'running_mean': array([-0.08286335, -0.07727974,  0.08525366, -0.12282139, -0.2330962 ,
       -0.13138105, -0.03352845,  0.04514041, -0.06611079, -0.12770451,
       -0.03460573,  0.11581767, -0.04147041, -0.04217178,  0.07648135,
        0.04472147, -0.03398877,  0.26535478, -0.07140276, -0.1518474 ]), 'running_var': array([0.00071735, 0.01194891, 0

{'mode': 'train', 'running_mean': array([-0.08286335, -0.07727974,  0.08525366, -0.12282139, -0.2330962 ,
       -0.13138105, -0.03352845,  0.04514041, -0.06611079, -0.12770451,
       -0.03460573,  0.11581767, -0.04147041, -0.04217178,  0.07648135,
        0.04472147, -0.03398877,  0.26535478, -0.07140276, -0.1518474 ]), 'running_var': array([0.00071735, 0.01194891, 0.02134588, 0.00237688, 0.00552786,
       0.00140453, 0.00142502, 0.0182396 , 0.0021617 , 0.06670861,
       0.01578707, 0.02765941, 0.00023354, 0.00146228, 0.02689178,
       0.01527322, 0.01009503, 0.01312208, 0.01002459, 0.02365058])}
batchnorm
{'mode': 'train', 'running_mean': array([-0.08286335, -0.07727974,  0.08525366, -0.12282139, -0.2330962 ,
       -0.13138105, -0.03352845,  0.04514041, -0.06611079, -0.12770451,
       -0.03460573,  0.11581767, -0.04147041, -0.04217178,  0.07648135,
        0.04472147, -0.03398877,  0.26535478, -0.07140276, -0.1518474 ]), 'running_var': array([0.00071735, 0.01194891, 0.02134588,

batchnorm
{'mode': 'train', 'running_mean': array([-0.08286335, -0.07727974,  0.08525366, -0.12282139, -0.2330962 ,
       -0.13138105, -0.03352845,  0.04514041, -0.06611079, -0.12770451,
       -0.03460573,  0.11581767, -0.04147041, -0.04217178,  0.07648135,
        0.04472147, -0.03398877,  0.26535478, -0.07140276, -0.1518474 ]), 'running_var': array([0.00071735, 0.01194891, 0.02134588, 0.00237688, 0.00552786,
       0.00140453, 0.00142502, 0.0182396 , 0.0021617 , 0.06670861,
       0.01578707, 0.02765941, 0.00023354, 0.00146228, 0.02689178,
       0.01527322, 0.01009503, 0.01312208, 0.01002459, 0.02365058])}
batchnorm
{'mode': 'train', 'running_mean': array([-0.08286335, -0.07727974,  0.08525366, -0.12282139, -0.2330962 ,
       -0.13138105, -0.03352845,  0.04514041, -0.06611079, -0.12770451,
       -0.03460573,  0.11581767, -0.04147041, -0.04217178,  0.07648135,
        0.04472147, -0.03398877,  0.26535478, -0.07140276, -0.1518474 ]), 'running_var': array([0.00071735, 0.01194891, 0

batchnorm
{'mode': 'train', 'running_mean': array([-0.08286335, -0.07727974,  0.08525366, -0.12282139, -0.2330962 ,
       -0.13138105, -0.03352845,  0.04514041, -0.06611079, -0.12770451,
       -0.03460573,  0.11581767, -0.04147041, -0.04217178,  0.07648135,
        0.04472147, -0.03398877,  0.26535478, -0.07140276, -0.1518474 ]), 'running_var': array([0.00071735, 0.01194891, 0.02134588, 0.00237688, 0.00552786,
       0.00140453, 0.00142502, 0.0182396 , 0.0021617 , 0.06670861,
       0.01578707, 0.02765941, 0.00023354, 0.00146228, 0.02689178,
       0.01527322, 0.01009503, 0.01312208, 0.01002459, 0.02365058])}
batchnorm
{'mode': 'train', 'running_mean': array([-0.08286335, -0.07727974,  0.08525366, -0.12282139, -0.2330962 ,
       -0.13138105, -0.03352845,  0.04514041, -0.06611079, -0.12770451,
       -0.03460573,  0.11581767, -0.04147041, -0.04217178,  0.07648135,
        0.04472147, -0.03398877,  0.26535478, -0.07140276, -0.1518474 ]), 'running_var': array([0.00071735, 0.01194891, 0

batchnorm
{'mode': 'train', 'running_mean': array([-0.08286335, -0.07727974,  0.08525366, -0.12282139, -0.2330962 ,
       -0.13138105, -0.03352845,  0.04514041, -0.06611079, -0.12770451,
       -0.03460573,  0.11581767, -0.04147041, -0.04217178,  0.07648134,
        0.04472147, -0.03398877,  0.26535478, -0.07140277, -0.15184741]), 'running_var': array([0.00071735, 0.01194891, 0.02134588, 0.00237688, 0.00552786,
       0.00140453, 0.00142502, 0.0182396 , 0.0021617 , 0.06670861,
       0.01578707, 0.02765941, 0.00023354, 0.00146228, 0.02689178,
       0.01527322, 0.01009503, 0.01312208, 0.01002459, 0.02365058])}
batchnorm
{'mode': 'train', 'running_mean': array([-0.08286335, -0.07727974,  0.08525366, -0.12282139, -0.2330962 ,
       -0.13138105, -0.03352845,  0.04514041, -0.06611079, -0.12770451,
       -0.03460573,  0.11581767, -0.04147041, -0.04217178,  0.07648134,
        0.04472147, -0.03398877,  0.26535478, -0.07140277, -0.15184741]), 'running_var': array([0.00071735, 0.01194891, 0

# Batchnorm for deep networks
Run the following to train a six-layer network on a subset of 1000 training examples both with and without batch normalization.

In [ ]:
np.random.seed(231)
# Try training a very deep net with batchnorm
hidden_dims = [100, 100, 100, 100, 100]

num_train = 1000
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

weight_scale = 2e-2
bn_model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization='batchnorm')
model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization=None)


print('Solver with batch norm:')
bn_solver = Solver(bn_model, small_data,
                num_epochs=10, batch_size=50,
                update_rule='adam',
                optim_config={
                  'learning_rate': 1e-3,
                },
                verbose=True,print_every=20)
bn_solver.train()


print('\nSolver without batch norm:')
solver = Solver(model, small_data,
                num_epochs=10, batch_size=50,
                update_rule='adam',
                optim_config={
                  'learning_rate': 1e-3,
                },
                verbose=True, print_every=20)
solver.train()


Run the following to visualize the results from two networks trained above. You should find that using batch normalization helps the network to converge much faster.

In [ ]:
def plot_training_history(title, label, baseline, bn_solvers, plot_fn, bl_marker='.', bn_marker='.', labels=None):
    """utility function for plotting training history"""
    plt.title(title)
    plt.xlabel(label)
    bn_plots = [plot_fn(bn_solver) for bn_solver in bn_solvers]
    bl_plot = plot_fn(baseline)
    num_bn = len(bn_plots)
    for i in range(num_bn):
        label='with_norm'
        if labels is not None:
            label += str(labels[i])
        plt.plot(bn_plots[i], bn_marker, label=label)
    label='baseline'
    if labels is not None:
        label += str(labels[0])
    plt.plot(bl_plot, bl_marker, label=label)
    plt.legend(loc='lower center', ncol=num_bn+1) 

    
plt.subplot(3, 1, 1)
plot_training_history('Training loss','Iteration', solver, [bn_solver], \
                      lambda x: x.loss_history, bl_marker='o', bn_marker='o')
plt.subplot(3, 1, 2)
plot_training_history('Training accuracy','Epoch', solver, [bn_solver], \
                      lambda x: x.train_acc_history, bl_marker='-o', bn_marker='-o')
plt.subplot(3, 1, 3)
plot_training_history('Validation accuracy','Epoch', solver, [bn_solver], \
                      lambda x: x.val_acc_history, bl_marker='-o', bn_marker='-o')

plt.gcf().set_size_inches(15, 15)
plt.show()

# Batch normalization and initialization
We will now run a small experiment to study the interaction of batch normalization and weight initialization.

The first cell will train 8-layer networks both with and without batch normalization using different scales for weight initialization. The second layer will plot training accuracy, validation set accuracy, and training loss as a function of the weight initialization scale.

In [ ]:
np.random.seed(231)
# Try training a very deep net with batchnorm
hidden_dims = [50, 50, 50, 50, 50, 50, 50]
num_train = 1000
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

bn_solvers_ws = {}
solvers_ws = {}
weight_scales = np.logspace(-4, 0, num=20)
for i, weight_scale in enumerate(weight_scales):
  print('Running weight scale %d / %d' % (i + 1, len(weight_scales)))
  bn_model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization='batchnorm')
  model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization=None)

  bn_solver = Solver(bn_model, small_data,
                  num_epochs=10, batch_size=50,
                  update_rule='adam',
                  optim_config={
                    'learning_rate': 1e-3,
                  },
                  verbose=False, print_every=200)
  bn_solver.train()
  bn_solvers_ws[weight_scale] = bn_solver

  solver = Solver(model, small_data,
                  num_epochs=10, batch_size=50,
                  update_rule='adam',
                  optim_config={
                    'learning_rate': 1e-3,
                  },
                  verbose=False, print_every=200)
  solver.train()
  solvers_ws[weight_scale] = solver

In [ ]:
# Plot results of weight scale experiment
best_train_accs, bn_best_train_accs = [], []
best_val_accs, bn_best_val_accs = [], []
final_train_loss, bn_final_train_loss = [], []

for ws in weight_scales:
  best_train_accs.append(max(solvers_ws[ws].train_acc_history))
  bn_best_train_accs.append(max(bn_solvers_ws[ws].train_acc_history))
  
  best_val_accs.append(max(solvers_ws[ws].val_acc_history))
  bn_best_val_accs.append(max(bn_solvers_ws[ws].val_acc_history))
  
  final_train_loss.append(np.mean(solvers_ws[ws].loss_history[-100:]))
  bn_final_train_loss.append(np.mean(bn_solvers_ws[ws].loss_history[-100:]))
  
plt.subplot(3, 1, 1)
plt.title('Best val accuracy vs weight initialization scale')
plt.xlabel('Weight initialization scale')
plt.ylabel('Best val accuracy')
plt.semilogx(weight_scales, best_val_accs, '-o', label='baseline')
plt.semilogx(weight_scales, bn_best_val_accs, '-o', label='batchnorm')
plt.legend(ncol=2, loc='lower right')

plt.subplot(3, 1, 2)
plt.title('Best train accuracy vs weight initialization scale')
plt.xlabel('Weight initialization scale')
plt.ylabel('Best training accuracy')
plt.semilogx(weight_scales, best_train_accs, '-o', label='baseline')
plt.semilogx(weight_scales, bn_best_train_accs, '-o', label='batchnorm')
plt.legend()

plt.subplot(3, 1, 3)
plt.title('Final training loss vs weight initialization scale')
plt.xlabel('Weight initialization scale')
plt.ylabel('Final training loss')
plt.semilogx(weight_scales, final_train_loss, '-o', label='baseline')
plt.semilogx(weight_scales, bn_final_train_loss, '-o', label='batchnorm')
plt.legend()
plt.gca().set_ylim(1.0, 3.5)

plt.gcf().set_size_inches(15, 15)
plt.show()

## Inline Question 1:
Describe the results of this experiment. How does the scale of weight initialization affect models with/without batch normalization differently, and why?

## Answer:
[FILL THIS IN]


# Batch normalization and batch size
We will now run a small experiment to study the interaction of batch normalization and batch size.

The first cell will train 6-layer networks both with and without batch normalization using different batch sizes. The second layer will plot training accuracy and validation set accuracy over time.

In [41]:
def run_batchsize_experiments(normalization_mode):
    np.random.seed(231)
    # Try training a very deep net with batchnorm
    hidden_dims = [100, 100, 100, 100, 100]
    num_train = 1000
    small_data = {
      'X_train': data['X_train'][:num_train],
      'y_train': data['y_train'][:num_train],
      'X_val': data['X_val'],
      'y_val': data['y_val'],
    }
    print(small_data['X_train'].shape)
    n_epochs=10
    weight_scale = 2e-2
    batch_sizes = [5,10,50]
    lr = 10**(-3.5)
    solver_bsize = batch_sizes[0]
    
    print('No normalization: batch size = ',solver_bsize)
    model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization=None)
    solver = Solver(model, small_data,
                    num_epochs=n_epochs, batch_size=solver_bsize,
                    update_rule='adam',
                    optim_config={
                      'learning_rate': lr,
                    },
                    verbose=False)
    solver.train()
    
    bn_solvers = []
    for i in range(len(batch_sizes)):
        b_size=batch_sizes[i]
        print('Normalization: batch size = ',b_size)
        bn_model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization=normalization_mode)
        bn_solver = Solver(bn_model, small_data,
                        num_epochs=n_epochs, batch_size=b_size,
                        update_rule='adam',
                        optim_config={
                          'learning_rate': lr,
                        },
                        verbose=False)
        bn_solver.train()
        bn_solvers.append(bn_solver)
        
    
        
    return bn_solvers, solver, batch_sizes

batch_sizes = [5,10,50]
bn_solvers_bsize, solver_bsize, batch_sizes = run_batchsize_experiments('batchnorm')

(1000, 3, 32, 32)
No normalization: batch size =  5
Normalization: batch size =  5
Normalization: batch size =  10


KeyboardInterrupt: 

In [ ]:
plt.subplot(2, 1, 1)
plot_training_history('Training accuracy (Batch Normalization)','Epoch', solver_bsize, bn_solvers_bsize, \
                      lambda x: x.train_acc_history, bl_marker='-^', bn_marker='-o', labels=batch_sizes)
plt.subplot(2, 1, 2)
plot_training_history('Validation accuracy (Batch Normalization)','Epoch', solver_bsize, bn_solvers_bsize, \
                      lambda x: x.val_acc_history, bl_marker='-^', bn_marker='-o', labels=batch_sizes)

plt.gcf().set_size_inches(15, 10)
plt.show()

## Inline Question 2:
Describe the results of this experiment. What does this imply about the relationship between batch normalization and batch size? Why is this relationship observed?

## Answer:
[FILL THIS IN]


# Layer Normalization
Batch normalization has proved to be effective in making networks easier to train, but the dependency on batch size makes it less useful in complex networks which have a cap on the input batch size due to hardware limitations. 

Several alternatives to batch normalization have been proposed to mitigate this problem; one such technique is Layer Normalization [2]. Instead of normalizing over the batch, we normalize over the features. In other words, when using Layer Normalization, each feature vector corresponding to a single datapoint is normalized based on the sum of all terms within that feature vector.

[2] [Ba, Jimmy Lei, Jamie Ryan Kiros, and Geoffrey E. Hinton. "Layer Normalization." stat 1050 (2016): 21.](https://arxiv.org/pdf/1607.06450.pdf)

## Inline Question 3:
Which of these data preprocessing steps is analogous to batch normalization, and which is analogous to layer normalization?

1. Scaling each image in the dataset, so that the RGB channels for each row of pixels within an image sums up to 1.
2. Scaling each image in the dataset, so that the RGB channels for all pixels within an image sums up to 1.  
3. Subtracting the mean image of the dataset from each image in the dataset.
4. Setting all RGB values to either 0 or 1 depending on a given threshold.

## Answer:
[FILL THIS IN]


# Layer Normalization: Implementation

Now you'll implement layer normalization. This step should be relatively straightforward, as conceptually the implementation is almost identical to that of batch normalization. One significant difference though is that for layer normalization, we do not keep track of the moving moments, and the testing phase is identical to the training phase, where the mean and variance are directly calculated per datapoint.

Here's what you need to do:

* In `cs231n/layers.py`, implement the forward pass for layer normalization in the function `layernorm_backward`. 

Run the cell below to check your results.
* In `cs231n/layers.py`, implement the backward pass for layer normalization in the function `layernorm_backward`. 

Run the second cell below to check your results.
* Modify `cs231n/classifiers/fc_net.py` to add layer normalization to the `FullyConnectedNet`. When the `normalization` flag is set to `"layernorm"` in the constructor, you should insert a layer normalization layer before each ReLU nonlinearity. 

Run the third cell below to run the batch size experiment on layer normalization.

In [30]:
# Check the training-time forward pass by checking means and variances
# of features both before and after layer normalization   

# Simulate the forward pass for a two-layer network
np.random.seed(231)
N, D1, D2, D3 =4, 50, 60, 3
X = np.random.randn(N, D1)
W1 = np.random.randn(D1, D2)
W2 = np.random.randn(D2, D3)
a = np.maximum(0, X.dot(W1)).dot(W2)

print('Before layer normalization:')
print_mean_std(a,axis=1)

gamma = np.ones(D3)
beta = np.zeros(D3)
# Means should be close to zero and stds close to one
print('After layer normalization (gamma=1, beta=0)')
a_norm, _ = layernorm_forward(a, gamma, beta, {'mode': 'train'})
print_mean_std(a_norm,axis=1)

gamma = np.asarray([3.0,3.0,3.0])
beta = np.asarray([5.0,5.0,5.0])
# Now means should be close to beta and stds close to gamma
print('After layer normalization (gamma=', gamma, ', beta=', beta, ')')
a_norm, _ = layernorm_forward(a, gamma, beta, {'mode': 'train'})
print_mean_std(a_norm,axis=1)

Before layer normalization:
  means:  [-59.06673243 -47.60782686 -43.31137368 -26.40991744]
  stds:   [10.07429373 28.39478981 35.28360729  4.01831507]

After layer normalization (gamma=1, beta=0)
  means:  [-4.81096644e-16  0.00000000e+00  7.40148683e-17 -5.55111512e-16]
  stds:   [0.99999995 0.99999999 1.         0.99999969]

After layer normalization (gamma= [3. 3. 3.] , beta= [5. 5. 5.] )
  means:  [5. 5. 5. 5.]
  stds:   [2.99999985 2.99999998 2.99999999 2.99999907]



In [31]:
# Gradient check batchnorm backward pass
np.random.seed(231)
N, D = 4, 5
x = 5 * np.random.randn(N, D) + 12
gamma = np.random.randn(D)
beta = np.random.randn(D)
dout = np.random.randn(N, D)


ln_param = {}
fx = lambda x: layernorm_forward(x, gamma, beta, ln_param)[0]
fg = lambda a: layernorm_forward(x, a, beta, ln_param)[0]
fb = lambda b: layernorm_forward(x, gamma, b, ln_param)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
da_num = eval_numerical_gradient_array(fg, gamma.copy(), dout)
db_num = eval_numerical_gradient_array(fb, beta.copy(), dout)

_, cache = layernorm_forward(x, gamma, beta, ln_param)
dx, dgamma, dbeta = layernorm_backward(dout, cache)

#You should expect to see relative errors between 1e-12 and 1e-8


print('dx error: ', rel_error(dx_num, dx))
print('dgamma error: ', rel_error(da_num, dgamma))
print('dbeta error: ', rel_error(db_num, dbeta))

dx error:  1.4336161049967258e-09
dgamma error:  4.519489546032799e-12
dbeta error:  2.276445013433725e-12


# Layer Normalization and batch size

We will now run the previous batch size experiment with layer normalization instead of batch normalization. Compared to the previous experiment, you should see a markedly smaller influence of batch size on the training history!

In [ ]:
ln_solvers_bsize, solver_bsize, batch_sizes = run_batchsize_experiments('layernorm')

plt.subplot(2, 1, 1)
plot_training_history('Training accuracy (Layer Normalization)','Epoch', solver_bsize, ln_solvers_bsize, \
                      lambda x: x.train_acc_history, bl_marker='-^', bn_marker='-o', labels=batch_sizes)
plt.subplot(2, 1, 2)
plot_training_history('Validation accuracy (Layer Normalization)','Epoch', solver_bsize, ln_solvers_bsize, \
                      lambda x: x.val_acc_history, bl_marker='-^', bn_marker='-o', labels=batch_sizes)

plt.gcf().set_size_inches(15, 10)
plt.show()

## Inline Question 4:
When is layer normalization likely to not work well, and why?

1. Using it in a very deep network
2. Having a very small dimension of features
3. Having a high regularization term


## Answer:
[FILL THIS IN]
